### doc_entity
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E ENTIDADES NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.  

AS ENTIDADES A SEREM TRABALHADAS SÃO:
* PAÍSES
* PESSOAS

In [11]:
import os
import sys
import re
from IPython.display import clear_output
import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql
import pickle
import csv
import unicodedata

Especifica qual sistema operacional está sendo usado. Renato = Linux ; Marcelo = nt (Windows) e define variáveis a partir disso

In [12]:
if os.name == 'nt':
    encoding_type = 'utf-8'
    ssh_priv_key = 'C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa'
    ssh_user='marcelobribeiro'
    sql_user='marcelobribeiro'
    path_inputs = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/inputs/"
    #path_inputs = "C:/Users/MARCELO/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/inputs/"
    path_outputs = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/outputs/"
else:
    encoding_type = 'ISO-8859-1'
    ssh_priv_key = '/home/rsouza/.ssh/id_rsa'
    ssh_user='rsouza'
    sql_user='rsouza'
    path_inputs = "/home/rsouza/Documentos/text-learning-tools/inputs/"
    path_outputs = "/home/rsouza/Documentos/text-learning-tools/outputs/"

In [19]:
path_temp = 'C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-ignorecase'

In [13]:
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [14]:
countries = {}
country_names_file = path_inputs+"history-lab-country_table.csv"
with open(country_names_file, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=";")
    for row in reader:
        varnames = []
        en_country = row[0]
        pt_country = row[1]
        en_city = row[2]
        other_country = row[3]
        pt_city = row[4]
        other_city = row[5]
        pt_country_alt = strip_accents(pt_country)
        other_country_alt = strip_accents(other_country)
        pt_city_alt = strip_accents(pt_city)
        other_city_alt = strip_accents(other_city)
        varnames = [en_country]
        for var in [pt_country,en_city,other_country,pt_city,other_city,pt_country_alt,other_country_alt,pt_city_alt,other_city_alt]:
            if var not in varnames and var != "": varnames.append(var)
        #if en_country == 'Belarus': print(pt_country,'pt_country',en_city,'en_city',other_country_alt,'other_country_alt',pt_city,'pt_city',other_city,'other_city',pt_country_alt,'pt_country_alt',other_country,'other_country',pt_city,'pt_city',other_city,'other_city')
        countries[en_country] = varnames

In [15]:
countries

{'Afghanistan': ['Afghanistan',
  'Afeganistão',
  'Kabul',
  'Cabul',
  'Afeganistao'],
 'Aland Islands': ['Aland Islands', 'Ilhas Aland'],
 'Albania': ['Albania', 'Albânia', 'Tirana'],
 'Algeria': ['Algeria', 'Argélia', 'Algiers', 'Argelia'],
 'American Samoa': ['American Samoa', 'Samoa Americana', 'Pago Pago'],
 'Andorra': ['Andorra', 'Andorra la Vella'],
 'Angola': ['Angola', 'Luanda'],
 'Anguilla': ['Anguilla', 'The Valley'],
 'Antarctica': ['Antarctica',
  'Antártica',
  'Antártida',
  'Antartica',
  'Antartida'],
 'Antigua and Barbuda': ['Antigua and Barbuda',
  'Antígua e Barbuda',
  "St. John's",
  'Antigua e Barbuda'],
 'Argentina': ['Argentina', 'Buenos Aires'],
 'Armenia': ['Armenia', 'Armênia', 'Yerevan'],
 'Aruba': ['Aruba', 'Oranjestad'],
 'Australia': ['Australia', 'Austrália', 'Canberra'],
 'Austria': ['Austria', 'Áustria', 'Vienna', 'Viena'],
 'Azerbaijan': ['Azerbaijan', 'Azerbaijão', 'Baku', 'Azerbaijao'],
 'Bahrain': ['Bahrain', 'Bahrein', 'Manama'],
 'Bangladesh':

In [16]:
doc_id_list_file = path_outputs+'/doc_id_list.pkl'
doc_id_list = pickle.load(open(doc_id_list_file, 'rb'))

In [17]:
amb_alt_names_list = ['tirana','bermuda','sofia','santiago','geórgia','reunião','peru']

In [ ]:
percentil = int(len(doc_id_list)/100)
person_doc_list = []
extraction_validation_list = []
country_doc_list = []
country_id = 1 #test

'''itera nos documentos'''
for file in doc_id_list:
    name_validation = 'no' # set to 'yes' if name is present on document
    doc_id = re.sub('AAS_mre_(.*)\.txt', r'\1', file)
    
    '''mede percentual de conclusão da tarefa'''
    count_file = doc_id_list.index(file)
    if count_file % 100 == 0: 
        clear_output()
        print(int((count_file)/percentil),'% done')
            
    '''cria tags usando o 'palavras' e manipula texto'''
    #with open(os.path.join(path_inputs,file), 'r') as f:
    with open(os.path.join(path_temp,file), 'r', encoding='utf-8') as f:
        text = f.read()
        text_lower = text.lower()
        
        '''itera em países'''
        for country,alt_names in countries.items():      
            name = country
            freq_count = 0

            for alt_name in alt_names:
                alt_name_lower = alt_name.lower()
                country_subfreq = text_lower.count(alt_name_lower)
                freq_count += country_subfreq

                if alt_name_lower in amb_alt_names_list:
                    if alt_name_lower == 'sofia':
                        semantic_tag = '<hum>'
                    if alt_name_lower == 'bermuda':
                        semantic_tag = '<cloH>'
                    if alt_name_lower == 'santiago':
                        semantic_tag = '<hum>'
                    if alt_name_lower == 'geórgia':
                        semantic_tag = '<hum>'
                    if alt_name_lower == 'reunião':
                        semantic_tag = '<occ>'
                    if alt_name_lower == 'peru':
                        semantic_tag = '<Aorn>'
                    if alt_name_lower == 'tirana':
                        semantic_tag = '<jh>'
                    tag_checklist = []
                    #tagger = palavras_tagger(text)
                    #for i in tagger[0]:
                    #    tag_definition = i[3]
                    #    if semantic_tag in tag_definition:
                    #        tag = i[1]
                    #        tag = re.sub('[\[\]]', '', tag)
                    #        tag = re.sub('\=', ' ', tag)
                    #        tag = tag.lower()
                    #        tag_checklist.append(tag)
                    #for tag_check in tag_checklist:
                    #    tag_check_split = tag_check.split()
                    #    for i in tag_check.split():
                    #        if i == alt_name: 
                    #            freq_count -= 1
                    
            freq_count_test1 = freq_count # for further analysis

            '''se nome ocorrer no documento e for entidade correta, adiciona em lista'''
            if freq_count > 0:
                country_doc = [country_id, doc_id, freq_count]
                country_doc_list.append(country_doc)


In [30]:
country_doc_list

[[1, 'ag_1973.11.20_doc_I-8', 1],
 [1, 'ag_1973.11.20_doc_I-8', 2],
 [1, 'ag_1973.11.20_doc_I-8', 3],
 [1, 'ag_1973.11.20_doc_I-9', 1],
 [1, 'ag_1973.11.20_doc_I-9', 3],
 [1, 'ag_1973.11.20_doc_I-9', 2],
 [1, 'ag_1973.11.20_doc_I-9', 4],
 [1, 'ag_1973.11.20_doc_I-9', 1],
 [1, 'ag_1973.11.20_doc_I-11', 1],
 [1, 'ag_1973.11.20_doc_I-11', 1],
 [1, 'ag_1973.11.20_doc_I-13', 3],
 [1, 'ag_1973.11.20_doc_I-13', 1],
 [1, 'ag_1973.11.20_doc_I-13', 2],
 [1, 'ag_1973.11.20_doc_I-14', 1],
 [1, 'ag_1973.11.20_doc_I-14', 1],
 [1, 'ag_1973.11.20_doc_I-15', 1],
 [1, 'ag_1973.11.20_doc_I-15', 1],
 [1, 'ag_1973.11.20_doc_I-15', 1],
 [1, 'ag_1973.11.20_doc_I-16', 2],
 [1, 'ag_1973.11.20_doc_I-17', 2],
 [1, 'ag_1973.11.20_doc_I-17', 1],
 [1, 'ag_1973.11.20_doc_I-17', 2],
 [1, 'ag_1973.11.20_doc_I-17', 1],
 [1, 'ag_1973.11.20_doc_I-17', 1],
 [1, 'ag_1973.11.20_doc_I-19', 1],
 [1, 'ag_1973.11.20_doc_I-19', 1],
 [1, 'ag_1973.11.20_doc_I-20', 2],
 [1, 'ag_1973.11.20_doc_I-21', 1],
 [1, 'ag_1973.11.20_doc_I-21

In [ ]:
count = 0

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.146', 22),
                        ssh_private_key=ssh_priv_key,
                        ssh_private_key_password=pass_sshkey,
                        ssh_username=ssh_user,
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user=sql_user,
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    OBS: o ID não é VARCHAR!
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

    #cur.execute("DROP TABLE IF EXISTS countries")
    #cur.execute('''CREATE TABLE IF NOT EXISTS countries
    #           (id VARCHAR(128) PRIMARY KEY, name VARCHAR(128)
    #           DEFAULT NULL)
    #           ENGINE=MyISAM DEFAULT CHARSET='utf8';''')
    
    cur.execute("DROP TABLE IF EXISTS country_doc")
    cur.execute('''CREATE TABLE IF NOT EXISTS country_doc
               (country_id VARCHAR(128), doc_id VARCHAR(31), country_count INT(11)
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')
    
    cur.execute("SELECT * FROM CPDOC_AS.docs WHERE main_language = 'pt' AND (readability > 0.4 OR readability = -1) ")  
    
    text_data = cur.fetchall()
    
    for country,values in countries.items():
        
        ### mede etapa da iteração. Número de etapas = 278 ###
        clear_output()
        print("Mining country number: ",count)
        
        for row_text in text_data:
            text = row_text[4]
            doc_id = row_text[0]
                        
            country_count = 0
            for name in values:
                country_subfreq = text.count(name)
                country_count += country_subfreq
            
            if country_count > 0: 
                query = "INSERT INTO country_doc VALUES (%s, %s, %s)"
                cur.execute(query, (country_id, doc_id, country_count))
        
        #query = "INSERT INTO countries VALUES (%s, %s)"
        #cur.execute(query, (country_id, country))
        
        count += 1 

In [7]:
response = re.search('mand', 'Mandy Pande', re.IGNORECASE)

In [8]:
response

<_sre.SRE_Match object; span=(0, 4), match='Mand'>

In [9]:
word = 'santiago de compostela'
word.split()

['santiago', 'de', 'compostela']